In [ ]:

!pip install transformers accelerate torch langchain elasticsearch sentence-transformers
!pip install torch --index-url https://download.pytorch.org/whl/cu118


In [ ]:

from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer

# Elasticsearch bağlantısı
es = Elasticsearch("http://localhost:9200")

# Embedding modeli
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def get_embedding(text):
    """Verilen metni vektör haline getirir."""
    return embedding_model.encode(text).tolist()


In [ ]:

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# LLaMA 3.1 8B modelini GPU'ya yükle
model_id = "meta-llama/Meta-Llama-3-8B"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,  
    device_map="auto"
)

def llama_generate(prompt, max_length=256):
    """LLaMA 3.1 modelini kullanarak metin üretir."""
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    output = model.generate(input_ids, max_length=max_length)
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [ ]:

from langchain.memory import ConversationSummaryMemory

# Parametrik geçmiş yönetimi (son 3 mesaj)
def get_memory(n=3):
    """Son N mesajı hatırlayan session bazlı sohbet hafızası."""
    return ConversationSummaryMemory(
        memory_key="chat_history",
        max_token_limit=n  
    )

memory = get_memory(n=3)


In [ ]:

import re
from langchain.tools import Tool

# Kampanya Kodunu Çıkarma
def extract_campaign_code(query):
    """Metin içindeki kampanya kodunu çıkarır."""
    match = re.search(r'\b\d{5,}\b', query)
    return match.group() if match else None

extract_campaign_code_tool = Tool(
    name="Extract Campaign Code",
    func=extract_campaign_code,
    description="Extracts a campaign code from the user query if available."
)

# Kampanya Koduna Göre Elasticsearch Arama
def search_campaign_by_code(campaign_no):
    """Kampanya kodu ile Elasticsearch'te arama yapar."""
    response = es.search(index="campaigns", body={"query": {"match": {"campaign_no": campaign_no}}})
    results = response.get("hits", {}).get("hits", [])
    return results[0]["_source"] if results else None

search_campaign_code_tool = Tool(
    name="Search Campaign by Code",
    func=search_campaign_by_code,
    description="Searches a campaign by its campaign_no field in Elasticsearch."
)

# Kampanya Başlığı ile Elasticsearch Embedding Arama
def search_campaign_by_header(query, top_n=3):
    """Campaign_header alanında embedding vektör ile arama yapar."""
    query_vector = get_embedding(query)
    
    script_query = {
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": "cosineSimilarity(params.query_vector, 'embedding') + 1.0",
                "params": {"query_vector": query_vector}
            }
        }
    }

    response = es.search(index="campaigns", body={"query": script_query, "size": top_n})
    results = response.get("hits", {}).get("hits", [])
    return [{"campaign_no": res["_source"]["campaign_no"], "campaign_header": res["_source"]["campaign_header"]} for res in results] if results else []

search_campaign_header_tool = Tool(
    name="Search Campaign by Header",
    func=search_campaign_by_header,
    description="Finds top N relevant campaigns using embedding search in the campaign_header field."
)

# Kampanya İçeriğinden LLaMA Yanıtı Üretme
def generate_campaign_response(campaign):
    """Kampanya bilgilerini kullanarak LLM tabanlı yanıt oluşturur."""
    prompt = f"Kullanıcı şu kampanya hakkında bilgi istiyor: {campaign['campaign_header']}. Açıklama: {campaign['campaign_description']}. Cevap oluştur."
    return llama_generate(prompt)

generate_campaign_response_tool = Tool(
    name="Generate Campaign Response",
    func=generate_campaign_response,
    description="Generates an AI-powered response based on campaign details."
)


In [ ]:

from langchain.agents import initialize_agent, AgentType

# Agent oluştur
agent = initialize_agent(
    tools=[
        extract_campaign_code_tool,
        search_campaign_code_tool,
        search_campaign_header_tool,
        generate_campaign_response_tool
    ],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)


In [ ]:

def chatbot(query):
    """Kullanıcının doğal konuşmasını ve bağlamı yönetir."""

    chat_history = memory.load_memory_variables({}).get('chat_history', [])
    if len(chat_history) > 3:
        chat_history = chat_history[-3:]

    campaign_no = extract_campaign_code(query)

    if campaign_no:
        campaign = search_campaign_by_code(campaign_no)
        if campaign:
            return generate_campaign_response(campaign)
        return f"Kampanya kodu {campaign_no} bulunamadı."

    results = search_campaign_by_header(query, top_n=3)

    if results:
        options_text = "\n".join([f"{res['campaign_no']} - {res['campaign_header']}" for res in results])
        return f"Aşağıdaki kampanyaları buldum. Seçmek istediğiniz kampanyanın başlığını ya da kodunu yazabilirsiniz:\n\n{options_text}"
    
    return "Kampanya bulunamadı."


In [ ]:

query1 = "Elektronik kampanyaları var mı?"
print(chatbot(query1))

query2 = "62222"
print(chatbot(query2))

query3 = "Elektronik eşyalarda büyük fırsatlar"
print(chatbot(query3))
